In [2]:
# GLOBAL
import os
import pandas as pd
import numpy as np
import tiktoken
from uuid import uuid4
# from tqdm import tqdm
from dotenv import load_dotenv
from tqdm.autonotebook import tqdm


# LANGCHAIN
import langchain
from langchain.llms import OpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate

# VECTOR STORE
import pinecone
from pinecone import Pinecone, ServerlessSpec

# AGENTS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, Tool, AgentType
from langchain.agents.react.agent import create_react_agent
from langchain import hub 

C:\Users\SONIC LAPTOPS\AppData\Local\Temp\ipykernel_3352\3698905458.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
apis  = load_dotenv()

# Access the environment variables
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY')
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

In [4]:
import google.generativeai as genai
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

Modal  = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
Response =  Modal.invoke("Hello, how are you?")
print(Response.content)

As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you might have! How can I help you today? 



In [7]:
promte = hub.pull("hwchase17/react")
print(promte.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [8]:
# Set prompt template

template= '''
          Answer the following questions as best you can. You have access to the following tools:

          {tools}

          Use the following format:

          Question: the input question you must answer
          Thought: you should always think about what to do
          Action: the action to take, should be one of [{tool_names}]. Always look first in Pinecone Document Store if not then use the tool
          Action Input: the input to the action
          Observation: the result of the action
          ... (this Thought/Action/Action Input/Observation can repeat 2 times)
          Thought: I now know the final answer
          Final Answer: the final answer to the original input question

          Begin!

          Question: {input}
          Thought:{agent_scratchpad}
          '''

prompt = PromptTemplate.from_template(template)

In [20]:
%%capture
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [23]:
%%capture
!pip install langchain-googledrive

In [9]:
os.environ["GOOGLE_ACCOUNT_FILE"] = "./credentials.json"
# Set up tools and agent
import os
from langchain_googledrive.utilities.google_drive import GoogleDriveAPIWrapper
from langchain_googledrive.tools.google_drive.tool import GoogleDriveSearchTool
from langchain.agents import initialize_agent, AgentType 
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily = TavilySearchResults(max_results=10, tavily_api_key=TAVILY_API_KEY)

# GoogleMail = GmailToolkit()


folder_id = "root"
GoogleDrive = GoogleDriveSearchTool(
    api_wrapper=GoogleDriveAPIWrapper(
        folder_id=folder_id,
        num_results=2,
        template="gdrive-query-in-folder",  # Search in the body of documents
    )
)
tools = [   
     
    Tool(
        name="Tavily",
        func=tavily.run,
        description="Use this to lookup information from Tavily",
    ),
    Tool(
        name="Google Drive",
        func=GoogleDrive.run,
        description="Use this to lookup information from Google Drive",
    ),

]

# # Add Gmail tools individually
# gmail_tools = GoogleMail.get_tools()
# for tool in gmail_tools:
#     tools.append(
#         Tool(
#             name=tool.__class__.__name__,
#             func=tool.run,
#             description='Used this to lookup information from Gmail',
#         )
#     )

# Create agent
agent = create_react_agent(Modal, tools, prompt)

agent_executor = AgentExecutor(
                        tools=tools,
                        agent=agent,
                        handle_parsing_errors=True,
                        verbose=True,
                    )

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=802591469297-9igthftlqvgk37sepgq90ci819demrfq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A63542%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=hmIAOXGEAY5l3LOaCHk4JVPmiskguI&access_type=offline


In [ ]:
pip install --upgrade requests urllib3

In [72]:
response = agent_executor.invoke({"input": "what is the rola of PROF. DR. HABIB UR REHMAN at UET ?"}) 



> Entering new AgentExecutor chain...
Question: what is the role of PROF. DR. HABIB UR REHMAN at UET ?
Thought: I need to find information about PROF. DR. HABIB UR REHMAN at UET. I can try looking for this information on the UET website.
Action: Tavily
Action Input: "UET website"[{'url': 'https://web.uettaxila.edu.pk/', 'content': 'Explore the official website of UET Taxila, a leading engineering university in Pakistan. Find out about its departments, admissions, fee, and more.'}, {'url': 'https://civil.uet.edu.pk/', 'content': 'Learn about the oldest and largest civil engineering department in Pakistan, offering undergraduate and postgraduate programs at UET Lahore. Explore the news, events, and achievements of the department and its faculty.'}, {'url': 'https://uet.edu.pk/home/index.html?reloaded&sort=date_desc&page=1', 'content': 'In UET, 738 International Students are enrolled, and they are provided perfect place to gain knowledge related to practical approach of communication st

In [12]:
response = agent_executor.invoke({"input": "How Many files in my Google Drive?"})



> Entering new AgentExecutor chain...
Question: How Many files in my Google Drive?
Thought: I need to access Google Drive to count the number of files.
Action: Google Drive
Action Input: {'action': 'count_files'}

HttpError: <HttpError 400 when requesting https://www.googleapis.com/drive/v3/files?fields=nextPageToken%2C+files%28id%2C+name%2C+mimeType%2C+description%2C+webViewLink%2C+webContentLink%2C+owners%2FdisplayName%2C+shortcutDetails%2C+sha256Checksum%2C+modifiedTime%29&includeItemsFromAllDrives=false&pageSize=100&supportsAllDrives=true&q=+fullText+contains+%27%7B%27action%27%3A+%27count_files%27%7D%27+and+%27root%27+in+parents+and+trashed%3Dfalse+&alt=json returned "Invalid Value". Details: "[{'message': 'Invalid Value', 'domain': 'global', 'reason': 'invalid', 'location': 'q', 'locationType': 'parameter'}]">